In [39]:
import sys
sys.path.append('../')
import os

from pathlib import Path
import time
import numpy as np
import scipy.optimize
import pickle
import matplotlib.pyplot as plt

from py_diff_pd.common.common import ndarray, create_folder
from py_diff_pd.common.common import print_info, print_ok, print_error, print_warning
from py_diff_pd.common.grad_check import check_gradients
from py_diff_pd.common.display import export_gif
from py_diff_pd.core.py_diff_pd_core import StdRealVector
from py_diff_pd.env.soft_starfish_env_3d import SoftStarfishEnv3d
from py_diff_pd.common.project_path import root_path
from py_diff_pd.core.py_diff_pd_core import HexMesh3d, HexDeformable, StdRealVector
import py_diff_pd.common.hex_mesh as hex

In [40]:
input_dir = path = "/mnt/e/muscleCode/sample_muscle_data/starfish"
input_obj_path = os.path.join(input_dir, "starfish_demo.obj")
hex_bin_path = os.path.join(input_dir, "starfish_demo.bin")
hex_obj_path = os.path.join(input_dir, "starfish_hex.obj")
dx = 0.18
origin = ndarray([0, 0, 0])

In [41]:
# abs path
input_obj_path = '/mnt/e/muscleCode/sample_muscle_data/starfish/starfish_demo.obj'

In [44]:
voxels = hex.voxelize(input_obj_path, dx)

loaded as trimesh
[5 0 5] dimensions


In [45]:
voxel_indices, vertex_flags = hex.generate_hex_mesh(voxels, dx, origin, hex_bin_path)

IndexError: tuple index out of range

In [ ]:
hex_object = HexMesh3d()
hex_object.Initialize(hex_bin_path)

In [ ]:
hex.hex2obj(hex_object, hex_obj_path)